# Imports

In [1]:
import pickle
from config import *
import pandas as pd

# Read + process Data

In [2]:
with open(BaselinePath, 'rb') as f:
    raw_dataset_dict = pickle.load(f)

with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)
    

In [3]:
categories_per_indent_dict ={key: categories_per_indent_dict[key] for key in categories_per_indent_dict.keys() if key in [0,1,2,3]}
cat_ids = list(categories_per_indent_dict.values())

flat_cat_id = [item for sublist in cat_ids for item in sublist]

categories = []
for cat_id in flat_cat_id:
    categories.append(category_id_to_name_dict[cat_id])

In [4]:
raw_dataset_dict = {key : val for key, val in raw_dataset_dict.items() if key not in exclude_categories}

In [5]:
raw_dataset_dict = {key : val for key, val in raw_dataset_dict.items() if key in categories}

In [6]:
def train_test_split(dic: dict, year: int) -> dict:
    train = {}
    test = {}
    all_categories = []
    for key in dic.keys():
        category_df = dic[key]
        all_categories.append(category_df)
    df = pd.concat(all_categories)
    train = df[df['Year'] <= year]
    test = df[df['Year'] > year]
    return train, test

raw_train_data, raw_test_data = train_test_split(raw_dataset_dict, year=Year)

In [7]:
def get_relevant_features(df, drop_columns: list) -> dict:
    processed_df = df.copy()
    processed_df = processed_df.drop(columns=drop_columns)
    processed_df.dropna(inplace=True)
    return processed_df

In [8]:
drop_columns = ['Category_id', 'Category', 'Year', 'Date', 'Price', 'Indent', 'Weight', 'Parent', 'Parent_ID']

train_data = get_relevant_features(raw_train_data, drop_columns)

test_data = get_relevant_features(raw_test_data, drop_columns)

In [9]:
with open('/Users/mvilenko/Desktop/CPI_HRNN - version 2.0/mayas_project/SGRU/data/train_dataset.pickle', 'wb') as handle:
    pickle.dump(train_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/Users/mvilenko/Desktop/CPI_HRNN - version 2.0/mayas_project/SGRU/data/test_dataset.pickle', 'wb') as handle:
    pickle.dump(test_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
train_data.shape

(17828, 14)

In [11]:
test_data.shape

(2356, 14)